In [1]:
import pandas as pd
import numpy as np
from scipy import stats
# from pyhive import presto
from requests.auth import HTTPBasicAuth
import requests
import time
import json
from pandas.io.json import json_normalize

In [2]:
# !conda install pandas=1.0.5

In [3]:
# !pip install pyhive
import os

cdir = os.getcwd()
print(cdir)
# os.chdir("C:\Users\yeude.lim\Desktop\Workk\workbook_123AA\Anaconda_project\GrabFood_code")

D:\Users\Yeu D\OneDrive - Nanyang Technological University\Coding python\Jupyter Lab Processing\20200628


# Define date and time requires

In [8]:
# input startdate, enddate, starttime and endtime here

startDate = '2020-06-27' # follow this format : 'yyyy-mm-dd'
endDate = '2020-06-28'
startTime = '01' # international time input, i.e. 08 09 13 15
endTime = '23'

# dateDate = ['2020-06-15'
#             ,'2020-06-16'
#             ,'2020-06-17'
#             ,'2020-06-18'
#             ,'2020-06-19'
#             ,'2020-06-20'
#             ,'2020-06-21']
# dateTime = ['09','10','11','12','13','14','15','16','17','18','19','20','21']

# convert string into datetime for better API query

In [9]:
# converting string into datetime..

from datetime import timedelta, date
from datetime import datetime

def daterange(startDate, endDate):
    for n in range(int ((endDate - startDate).days)):
        yield startDate + timedelta(n)
    yield endDate

def timerange(startTime, endTime):
    for n in range(int ((endTime - startTime).seconds/3600)):
#         print(int((endTime - startTime).seconds/3600))
        yield startTime + timedelta(seconds = n*3600)
    yield endTime

startDate = datetime.strptime(startDate, "%Y-%m-%d")
endDate = datetime.strptime(endDate, "%Y-%m-%d")

startTime = datetime.strptime(startTime,"%H")
endTime = datetime.strptime(endTime,"%H")

# initialize
dateDate = []
dateTime = []

for single_date in daterange(startDate, endDate):
#     print(single_date.strftime("%Y-%m-%d"))
    dateDate.append(single_date.strftime("%Y-%m-%d"))
    
for single_time in timerange(startTime, endTime):
#     print(single_time.strftime("%H"))
    dateTime.append(single_time.strftime("%H"))
    
print(dateDate)
print(dateTime)

['2020-06-27', '2020-06-28']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


# Get API response from the gov URL

- This section query the weather data based on the date and time defined above
- Loop based until 'All is well' occured
- i.e. 2020-06-15, 09 ~ 21; 2020-06-16, 09 ~ 21 and etc

In [10]:
# run loop
df_week_v0 = {} # initialize

for i in range(len(dateDate)):
    for j in range(len(dateTime)):
        weather_api = "https://api.data.gov.sg/v1/environment/rainfall?date_time={}T{}%3A00%3A00&date={}".format(dateDate[i],dateTime[j],dateDate[i])
        result = requests.get(weather_api)
        # df_weather_1 = pd.read_json(result.text)
        # df_weather_1
        data = json.loads(result.text)
        df1 = json_normalize(data['metadata']['stations'])
        df1['dateandtime'] = dateDate[i]+'_'+ dateTime[j]
        
        df2 = json_normalize(data['items'][0]['readings'])
        datetime_current = json_normalize(data['items'])['timestamp']
#         df2['date_and_time'] = pd.json_normalize(data['items'])[['timestamp']]
        df2['date_and_time'] = datetime_current
        df2.date_and_time = df2.date_and_time.astype('datetime64[ns]')
        datetimeanddate = dateDate[i] + '_' + dateTime[j]
#         dict_a = {dateDate[i]:pd.merge(left=df1, right=df2, left_on='device_id', right_on='station_id')}
        df_week_v0[datetimeanddate] = pd.merge(left=df1, right=df2, left_on='device_id', right_on='station_id')
        
    print('Date: ' + dateDate[i] + '...done!')

print('All is well!')
# data['metadata']

Date: 2020-06-27...done!
Date: 2020-06-28...done!
All is well!


# Single Query weather data here
Use only if necessary for single check

In [11]:
# query_date = '2020-06-24' # input date for weather query
# query_time = '08' # international time (00-23), i.e. 09, 13, 15, 21
# date_and_time = query_date + '_' + query_time
# df_week[date_and_time].sort_values(by='value',ascending=False).head(15)

In [12]:
# to save into excel, limited to 2 days only

import pandas as pd

excel_filename = dateDate[0] + '_weather.xlsx' # The excel file name
writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

for i in range(len(dateDate)):
    for j in range(len(dateTime)):
        datetimeanddate = dateDate[i] + '_' + dateTime[j]

        df_week_v0[datetimeanddate].sort_values(by='value',ascending=False).to_excel(writer, sheet_name=datetimeanddate, index=False)
        
writer.save()

# ----- Plot in map

1. Bigger size of dots = Heavier raining condition
2. Color is arbitrary, right now, same rain condition comes with same colour
2. input query date and time
3. change color if needed

In [13]:
# color selection
#     'red',
#     'blue',
#     'gray',
#     'darkred',
#     'lightred',
#     'orange',
#     'beige',
#     'green',
#     'darkgreen',
#     'lightgreen',
#     'darkblue',
#     'lightblue',
#     'purple',
#     'darkpurple',
#     'pink',
#     'cadetblue',
#     'lightgray',
#     'black'

In [14]:
# !pip install folium       # comment this out if need to install folium

# import
import folium
from folium import plugins
from folium import features

print('Folium imported!')

# define the world map centered around Canada with a low zoom level
sg_map = folium.Map(location = [1.35, 103.82], zoom_start=12, tiles='CartoDB positron')

Folium imported!


## Input date and time at this cell

In [21]:
# For input:
# -----------------------------------------------
query_date = '2020-06-28'    # input date for weather query (must be within the date you have pulled from API), i.e. '2020-06-22'
query_time = '13'            # international time (00-23), i.e. 09, 13, 15, 21
# -----------------------------------------------

In [22]:
date_and_time = query_date + '_' + query_time

# Other map type selection: tiles='Stamen Terrain','Stamen Toner', 'Stamen Watercolor', “CartoDB positron”, "CartoDB dark_matter"
sg_lat = 1.35
sg_lon = 103.82
sg_map = folium.Map(location = [sg_lat, sg_lon], zoom_start=12, tiles='CartoDB dark_matter')


for lat, lng, val, place, time in zip(df_week_v0[date_and_time]['location.latitude']
                                , df_week_v0[date_and_time]['location.longitude']
                                , df_week_v0[date_and_time].value
                                , df_week_v0[date_and_time].name
                                , df_week_v0[date_and_time].date_and_time):
#     label = place + ' ' + '{:.3f} mm/Hr'.format(val)
#     print(str(time))
    if val == 0.0000:
        label = place +' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'No_rain'
        radius = 2
        color = 'white'
        fill_color='white'
    elif (val > 0.0000 )and (val < 0.5000):
        label = place + ' ....... ' + '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Slight_rain'
        radius = 6
        color = 'pink'
        fill_color = 'pink'
    elif (val >= 0.5000 )and (val < 4.0000):
        label = place +' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Moderate_rain / Thick_Drizzle'
        radius = 9
        color = 'lightblue'
        fill_color = 'lightblue'
    elif (val >= 4.0000 )and (val < 8.0000):
        label = place + ' ....... ' + '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Heavy_rain / Moderate_Shower'
        radius = 13
        color = 'blue'
        fill_color = 'blue'
    else: #(val >= 8.0 )and (val < 50.0):
        label = place + ' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Very_Heavy_rain / Heavy_Shower'
        radius = 18
        color = 'red'
        fill_color = 'red'
    
#     folium.Marker(
    folium.CircleMarker(
        [lat, lng],
        radius= radius, #5, # define how big you want the circle markers to be
        color = color, #'blue',
        fill=True,
        popup=label,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(sg_map)
    

# display world map
sg_map

In [23]:
# df_week # [date_and_time][df_week[date_and_time]['name'] == 'Kim Chuan Road']

# Weather in time series

# ---- Make an all in one table by using Append

In [25]:
# run loop

# # -----------------------------------------------
# query_date = '2020-06-24'    # input date for weather query (must be within the date you have pulled from API), i.e. '2020-06-22'
# query_time = '09'            # international time (00-23), i.e. 09, 13, 15, 21
# # -----------------------------------------------
# date_and_time = query_date + '_' + query_time

startDate = '2020-06-27' # follow this format : 'yyyy-mm-dd'
endDate = '2020-06-28'
startTime = '01' # international time input, i.e. 08 09 13 15
endTime = '23'

# converting string into datetime..

from datetime import timedelta, date
from datetime import datetime

def daterange(startDate, endDate):
    for n in range(int ((endDate - startDate).days)):
        yield startDate + timedelta(n)
    yield endDate

def timerange(startTime, endTime):
    for n in range(int ((endTime - startTime).seconds/3600)):
#         print(int((endTime - startTime).seconds/3600))
        yield startTime + timedelta(seconds = n*3600)
    yield endTime

startDate = datetime.strptime(startDate, "%Y-%m-%d")
endDate = datetime.strptime(endDate, "%Y-%m-%d")

startTime = datetime.strptime(startTime,"%H")
endTime = datetime.strptime(endTime,"%H")

# initialize
dateDate = []
dateTime = []

for single_date in daterange(startDate, endDate):
#     print(single_date.strftime("%Y-%m-%d"))
    dateDate.append(single_date.strftime("%Y-%m-%d"))
    
for single_time in timerange(startTime, endTime):
#     print(single_time.strftime("%H"))
    dateTime.append(single_time.strftime("%H"))
    
print(dateDate)
print(dateTime)

df_week = pd.DataFrame() # initialize

for i in range(len(dateDate)):
    for j in range(len(dateTime)):
        weather_api = "https://api.data.gov.sg/v1/environment/rainfall?date_time={}T{}%3A00%3A00&date={}".format(dateDate[i],dateTime[j],dateDate[i])
        result = requests.get(weather_api)
        
        date_object = '{}'.format(dateDate[i]) + 'T{}:00:00'.format(dateTime[j])
        data = json.loads(result.text)
        df1 = json_normalize(data['metadata']['stations'])
        df1['dateandtime'] = date_object #dateDate[i]+'_'+ dateTime[j]
        df1['dateandtime'] = df1['dateandtime'].astype('datetime64[ns]')
        df2 = json_normalize(data['items'][0]['readings'])
        datetime_current = json_normalize(data['items'])['timestamp']
#         df2['date_and_time'] = pd.json_normalize(data['items'])[['timestamp']]
#         df2['date_and_time'] = datetime_current
#         df2.date_and_time = df2.date_and_time.astype('datetime64[ns]')
        datetimeanddate = dateDate[i] + '_' + dateTime[j]
#         dict_a = {dateDate[i]:pd.merge(left=df1, right=df2, left_on='device_id', right_on='station_id')}
        df_week = df_week.append(pd.merge(left=df1, right=df2, left_on='device_id', right_on='station_id'), ignore_index=True)
        
    print('Date: ' + dateDate[i] + '...done!')

print(df_week.shape)
print('All is well!')
# data['metadata']
df_week.head()

['2020-06-27', '2020-06-28']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
Date: 2020-06-27...done!
Date: 2020-06-28...done!
(3432, 8)
All is well!


,id,device_id,name,location.latitude,location.longitude,dateandtime,station_id,value
0,S77,S77,Alexandra Road,1.2937,103.8125,2020-06-27 01:00:00,S77,0.0
1,S109,S109,Ang Mo Kio Avenue 5,1.3764,103.8492,2020-06-27 01:00:00,S109,0.0
2,S117,S117,Banyan Road,1.2560,103.6790,2020-06-27 01:00:00,S117,0.0
3,S90,S90,Bukit Timah Road,1.3191,103.8191,2020-06-27 01:00:00,S90,0.0
4,S61,S61,Chai Chee Street,1.3230,103.9217,2020-06-27 01:00:00,S61,0.0


In [26]:
# check info
df_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3432 entries, 0 to 3431
Data columns (total 8 columns):
id                    3432 non-null object
device_id             3432 non-null object
name                  3432 non-null object
location.latitude     3432 non-null float64
location.longitude    3432 non-null float64
dateandtime           3432 non-null datetime64[ns]
station_id            3432 non-null object
value                 3432 non-null float64
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 214.6+ KB


In [27]:
from folium.plugins import HeatMap
from folium import plugins

import datetime as dt

sg_lat = 1.35
sg_lon = 103.82

# ----- For single map check
# date_call = dates_list_str[0]
# print(date_call)

dates_list = [dt.datetime.strptime(str(date).split('.')[0], '%Y-%m-%dT%H:%M:%S') for date in df_week['dateandtime'].unique()]
dates_list_str = [date.strftime('%Y-%m-%d %H:%M') for date in dates_list]

# create map of Calgary using latitude and longitude values
sg_map = folium.Map(location=[sg_lat, sg_lon],  zoom_start=11, tiles='CartoDB dark_matter')

# Just adding a marker for fun
folium.Marker((sg_lat,sg_lon), popup="Singapore").add_to(sg_map)

# df_week['value_weight'] = df_week['value']*(df_week['value'] - df_week['value'].min())/(df_week['value'].max() - df_week['value'].min())-0.2
df_week['value_weight'] = df_week['value']-1
# ----- Uncomment this for single heatmap
# -----------------------------------
# HeatMap(data=df_week[['location.latitude','location.longitude' , 'value']].groupby(['location.latitude','location.longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=4).add_to(sg_map)
# HeatMap(data=df_week[df_week['dateandtime'] == date_call][['location.latitude','location.longitude' , 'value']].reset_index(drop=True).values.tolist(), radius=20, max_zoom=4).add_to(sg_map)

# ------- If you want to save the map 
# base_heatmap.save(outfile= truckheatMap.html)

# ----- Time series heatmap
heat_data = [[[row['location.latitude'],row['location.longitude'],row['value_weight']] for index, row in df_week[df_week['dateandtime'] == i].iterrows()] for i in df_week['dateandtime'].unique()]
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8, radius = 20, use_local_extrema =True, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, index=dates_list_str)
hm.add_to(sg_map)

# To show the map in jupyter
sg_map


In [29]:
date_and_time = query_date + '_' + query_time

# Other map type selection: tiles='Stamen Terrain','Stamen Toner', 'Stamen Watercolor', “CartoDB positron”, "CartoDB dark_matter"
sg_lat = 1.35
sg_lon = 103.82
sg_map_2 = folium.Map(location = [sg_lat, sg_lon], zoom_start=12, tiles='CartoDB dark_matter')

date_call = dates_list_str[6]
print(date_call)
for lat, lng, val, place, time in zip(df_week[df_week['dateandtime']==date_call]['location.latitude']
                                , df_week[df_week['dateandtime']==date_call]['location.longitude']
                                , df_week[df_week['dateandtime']==date_call].value
                                , df_week[df_week['dateandtime']==date_call].name
                                , df_week[df_week['dateandtime']==date_call].dateandtime):
#     label = place + ' ' + '{:.3f} mm/Hr'.format(val)
#     print(str(time))
    if val == 0.0000:
        label = place +' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'No_rain'
        radius = 2
        color = 'white'
        fill_color='white'
    elif (val > 0.0000 )and (val < 0.5000):
        label = place + ' ....... ' + '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Slight_rain'
        radius = 6
        color = 'pink'
        fill_color = 'pink'
    elif (val >= 0.5000 )and (val < 4.0000):
        label = place +' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Moderate_rain / Thick_Drizzle'
        radius = 9
        color = 'lightblue'
        fill_color = 'lightblue'
    elif (val >= 4.0000 )and (val < 8.0000):
        label = place + ' ....... ' + '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Heavy_rain / Moderate_Shower'
        radius = 13
        color = 'blue'
        fill_color = 'blue'
    else: #(val >= 8.0 )and (val < 50.0):
        label = place + ' ....... '+ '{:.3f}mm/Hr'.format(val) + ' ....... ' + 'Very_Heavy_rain / Heavy_Shower'
        radius = 18
        color = 'red'
        fill_color = 'red'
    
#     folium.Marker(
    folium.CircleMarker(
        [lat, lng],
        radius= radius, #5, # define how big you want the circle markers to be
        color = color, #'blue',
        fill=True,
        popup=label,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(sg_map_2)
    

# display world map
sg_map_2

2020-06-27 07:00


In [266]:
df_week.dateandtime.value_counts()

2020-06-21 10:00:00    73
2020-06-21 09:00:00    73
2020-06-25 10:00:00    72
2020-06-25 09:00:00    71
2020-06-22 10:00:00    71
2020-06-21 08:00:00    71
2020-06-22 09:00:00    70
2020-06-23 10:00:00    69
2020-06-24 10:00:00    69
2020-06-24 09:00:00    68
2020-06-22 08:00:00    68
2020-06-25 08:00:00    68
2020-06-23 09:00:00    68
2020-06-24 08:00:00    67
2020-06-23 08:00:00    66
Name: dateandtime, dtype: int64